In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from copy import deepcopy
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("flights_2019_cleansed.csv")

In [3]:
df = df.sort_values(by='fl_date')

In [4]:
df.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,year
1527950,3311391,2019-01-01,UA,3873,ZW,N447AW,3873,12217,HSV,"Huntsville, AL",13930,ORD,"Chicago, IL",1850,1847.0,-3.0,15.0,1902.0,2028.0,36.0,2045,2104.0,19.0,0,0,N,115.0,137.0,86.0,1,510,2019
1145224,3300803,2019-01-01,WN,1458,WN,N441WN,1458,12206,HRL,"Harlingen/San Benito, TX",12191,HOU,"Houston, TX",2000,1957.0,-3.0,6.0,2003.0,2043.0,15.0,2100,2058.0,-2.0,0,0,N,60.0,61.0,40.0,1,277,2019
486241,3298378,2019-01-01,AA,2561,AA,N860NN,2561,14635,RSW,"Fort Myers, FL",13930,ORD,"Chicago, IL",1732,1758.0,26.0,15.0,1813.0,1954.0,43.0,1955,2037.0,42.0,0,0,N,203.0,219.0,161.0,1,1120,2019
1386189,3309289,2019-01-01,B6,176,B6,N625JB,176,14843,SJU,"San Juan, PR",10529,BDL,"Hartford, CT",1333,1328.0,-5.0,23.0,1351.0,1635.0,7.0,1637,1642.0,5.0,0,0,N,244.0,254.0,224.0,1,1666,2019
1126643,3313381,2019-01-01,AA,2798,AA,N846NN,2798,13930,ORD,"Chicago, IL",14771,SFO,"San Francisco, CA",1700,1750.0,50.0,19.0,1809.0,2015.0,10.0,1954,2025.0,31.0,0,0,N,294.0,275.0,246.0,1,1846,2019


In [7]:
df.drop(['Unnamed: 0', 'wheels_off','wheels_on','taxi_in','taxi_out', 'tail_num', 'mkt_carrier_fl_num','cancelled','diverted', 'dup'], axis = 1, inplace = True)

In [8]:
df['fl_date'] = df['fl_date'].astype('datetime64[ns]')

day = df['fl_date'].dt.dayofyear
week = df['fl_date'].dt.week
df.insert(1, 'day', day)
df.insert(2, 'week', week)

In [10]:
df.tail()

,fl_date,day,week,mkt_unique_carrier,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,year
152231,2019-12-31,365,1,UA,EV,4458,15412,TYS,"Knoxville, TN",11618,EWR,"Newark, NJ",700,707.0,7.0,909,908.0,-1.0,129.0,121.0,90.0,1,631,2019
1609485,2019-12-31,365,1,AA,MQ,3856,15048,SUX,"Sioux City, IA",11298,DFW,"Dallas/Fort Worth, TX",1519,1512.0,-7.0,1739,1704.0,-35.0,140.0,112.0,98.0,1,657,2019
1148750,2019-12-31,365,1,AA,AA,1839,11278,DCA,"Washington, DC",11298,DFW,"Dallas/Fort Worth, TX",657,657.0,0.0,944,940.0,-4.0,227.0,223.0,197.0,1,1192,2019
1974633,2019-12-31,365,1,UA,UA,1205,12758,KOA,"Kona, HI",12892,LAX,"Los Angeles, CA",1405,1403.0,-2.0,2119,2158.0,39.0,314.0,355.0,321.0,1,2504,2019
1549156,2019-12-31,365,1,AA,AA,1098,15370,TUL,"Tulsa, OK",11298,DFW,"Dallas/Fort Worth, TX",918,911.0,-7.0,1039,1030.0,-9.0,81.0,79.0,48.0,1,237,2019


## Weather Features

In [12]:
origin_weather = pd.read_csv("weather.csv")

In [13]:
origin_weather.new_date = origin_weather.new_date.astype('datetime64[ns]')
origin_weather['year'] = origin_weather.new_date.dt.year

In [14]:
origin_weather = origin_weather[origin_weather['year'] == 2019]
origin_weather.drop('name', axis = 1, inplace= True)

In [15]:
origin_weather

,cloud_cover,precipitation,snow_depth,snow_fall,wind_speed,airport,new_date,year
138700,34,0.0,2.3,0.0,4.0,DEN,2019-01-01,2019
138701,0,0.0,2.1,0.0,4.7,DEN,2019-01-02,2019
138702,0,0.0,0.7,0.0,4.5,DEN,2019-01-03,2019
138703,2,0.0,0.0,0.0,5.3,DEN,2019-01-04,2019
138704,39,0.0,0.0,0.0,4.3,DEN,2019-01-05,2019
...,...,...,...,...,...,...,...,...
277395,65,0.0,2.2,0.0,11.2,XWA,2019-12-29,2019
277396,7,0.0,2.2,0.0,10.6,ISN,2019-12-30,2019
277397,7,0.0,2.2,0.0,10.6,XWA,2019-12-30,2019
277398,24,0.0,2.2,0.0,11.7,ISN,2019-12-31,2019


In [16]:
origin_weather['day'] = origin_weather.new_date.dt.dayofyear

In [17]:
origin_weather.rename(columns={'cloud_cover':'origin_cloud_cover', 'precipitation':'origin_precipitation', 'snow_depth':'origin_snow_depth', 'snow_fall':'origin_snow_fall', 'wind_speed':'origin_wind_speed', 'airport':'origin'}, inplace=True)

In [18]:
dest_weather = deepcopy(origin_weather)

In [19]:
dest_weather.rename(columns={'origin_cloud_cover':'dest_cloud_cover', 'origin_precipitation':'dest_precipitation', 'origin_snow_depth':'dest_snow_depth', 'origin_snow_fall':'dest_snow_fall', 'origin_wind_speed':'dest_wind_speed', 'origin':'dest'}, inplace = True)

In [20]:
dest_weather.head()

,dest_cloud_cover,dest_precipitation,dest_snow_depth,dest_snow_fall,dest_wind_speed,dest,new_date,year,day
138700,34,0.0,2.3,0.0,4.0,DEN,2019-01-01,2019,1
138701,0,0.0,2.1,0.0,4.7,DEN,2019-01-02,2019,2
138702,0,0.0,0.7,0.0,4.5,DEN,2019-01-03,2019,3
138703,2,0.0,0.0,0.0,5.3,DEN,2019-01-04,2019,4
138704,39,0.0,0.0,0.0,4.3,DEN,2019-01-05,2019,5


In [21]:
dest = list(df.dest.unique())
origin = list(df.origin.unique())

dest_weather = dest_weather[dest_weather['dest'].isin(dest)]
origin_weather = origin_weather[origin_weather['origin'].isin(origin)]

In [22]:
df = pd.merge(df, origin_weather, how='left', on=['day', 'origin'])
df = pd.merge(df, dest_weather, how='left', on = ['day','dest'])

In [23]:
df.drop(['new_date_x','year_y','new_date_y','year_x','year'], axis = 1, inplace = True)

## Shifting Negative arr_delay Values

In [24]:
df.loc[(df['arr_delay'] < 0), 'arr_delay'] = 0

## Moving Averages

In [25]:
# Origin airport moving average delay

df_ma_origin = df[['day', 'origin_airport_id', 'arr_delay']]

In [26]:
df_ma_origin = df_ma_origin.groupby(['origin_airport_id', 'day']).sum().reset_index()

In [27]:
df_ma_origin['origin_ma_arr_delay'] = df_ma_origin['arr_delay'].rolling(5).mean()

In [28]:
df_ma_origin.fillna(0, inplace=True)

In [29]:
df_ma_origin.drop('arr_delay', axis = 1, inplace=True)

In [30]:
df_ma_dest = df[['day', 'arr_delay', 'dest_airport_id']]
df_ma_dest = df_ma_dest.groupby(['dest_airport_id', 'day']).sum().reset_index()

df_ma_dest['dest_ma_arr_delay'] = df_ma_dest['arr_delay'].rolling(5).mean()
df_ma_dest.fillna(0, inplace=True)
df_ma_dest.drop('arr_delay', axis = 1, inplace=True)

In [31]:
df = pd.merge(df, df_ma_dest, how='left', on=['dest_airport_id', 'day'])
df = pd.merge(df, df_ma_origin, how='left', on=['origin_airport_id', 'day'])

In [32]:
# Binning Arrival/Dep Times for later

time_bins = np.arange(0, 2500, 100)
labels=time_bins[:-1]

df['arr_time_binned'] = pd.cut(df['crs_arr_time'], bins = time_bins, labels=labels)
df['dep_time_binned'] = pd.cut(df['crs_dep_time'], bins = time_bins, labels = labels)

In [33]:
# Encoding carrier

from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
y = df.op_unique_carrier.values
label_encoder = enc.fit(y)
y = label_encoder.transform(y)

df['encoded_op_unique_carrier'] = y

In [52]:
# Creating an encoding dictionary so I can map values later

carrier_labels = df.op_unique_carrier.value_counts().index
carrier_codes = df['encoded_op_unique_carrier'].value_counts().index

carrier_dict = dict(zip(carrier_labels, carrier_codes))

In [64]:
# Loading in passenger data

df_passengers = pd.read_csv('df_passengers.csv')

In [65]:
df_passengers = df_passengers[df_passengers['unique_carrier'].isin(list(df.op_unique_carrier.unique()))]

In [66]:
df_passengers.rename(columns={'unique_carrier':'op_unique_carrier'}, inplace = True)
df_passengers.drop(['month','seats','Unnamed: 0'], axis = 1, inplace =True)

df_passengers = df_passengers.groupby(['op_unique_carrier', 'route']).sum().reset_index()

In [67]:
df_passengers.rename(columns={'op_unique_carrier':'encoded_op_unique_carrier'}, inplace=True)
df_passengers['encoded_op_unique_carrier'] = df_passengers.encoded_op_unique_carrier.map(carrier_dict)

df_passengers.head()

,encoded_op_unique_carrier,route,passengers
0,0,1013510397,17147.5
1,0,1013512478,330.0
2,0,1013512953,161.0
3,0,1013513230,39.0
4,0,1014610397,13313.0


In [68]:
# Creating routes in original df

df['route'] = df['origin_airport_id'].astype('str')  + df['dest_airport_id'].astype('str')

In [71]:
# To make the merge possible

df['encoded_op_unique_carrier'] = df.encoded_op_unique_carrier.astype('int32')
df_passengers['encoded_op_unique_carrier'] = df_passengers.encoded_op_unique_carrier.astype('int32')
df_passengers['route'] = df_passengers['route'].astype('str')

In [72]:
df = pd.merge(df, df_passengers, how='left', on=['encoded_op_unique_carrier', 'route'])

In [73]:
df.rename(columns={'passengers':'annual_passengers_route'}, inplace = True)

In [82]:
df_carrier_delay_origin = df[[
    'encoded_op_unique_carrier', 'origin_airport_id', 'arr_delay', 'day']]
df_carrier_delay_origin = df_carrier_delay_origin.groupby(
    ['encoded_op_unique_carrier', 'day', 'origin_airport_id']).sum().reset_index()

df_carrier_delay_origin['carrier_ma_arr_delay_origin'] = df_carrier_delay_origin['arr_delay'].rolling(
    5).mean()
df_carrier_delay_origin.fillna(0, inplace=True)
df_carrier_delay_origin.drop('arr_delay', axis=1, inplace=True)
df = pd.merge(df, df_carrier_delay_origin, how='left', on=[
              'encoded_op_unique_carrier', 'day', 'origin_airport_id'])

In [83]:
df_carrier_delay_dest = df[['encoded_op_unique_carrier','dest_airport_id','arr_delay','day']]
df_carrier_delay_dest = df_carrier_delay_dest.groupby(['encoded_op_unique_carrier', 'day', 'dest_airport_id']).sum().reset_index()

df_carrier_delay_dest['carrier_ma_arr_delay_dest'] = df_carrier_delay_dest['arr_delay'].rolling(5).mean()
df_carrier_delay_dest.fillna(0, inplace=True)
df_carrier_delay_dest.drop('arr_delay', axis = 1, inplace=True)
df = pd.merge(df, df_carrier_delay_dest, how='left', on=['encoded_op_unique_carrier', 'day', 'dest_airport_id'])

In [84]:
df.head()

,fl_date,day,week,mkt_unique_carrier,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,origin_cloud_cover,origin_precipitation,origin_snow_depth,origin_snow_fall,origin_wind_speed,dest_cloud_cover,dest_precipitation,dest_snow_depth,dest_snow_fall,dest_wind_speed,dest_ma_arr_delay,origin_ma_arr_delay,arr_time_binned,dep_time_binned,encoded_op_unique_carrier,route,annual_passengers_route,carrier_ma_arr_delay_x,carrier_ma_arr_delay_y,carrier_ma_arr_delay,carrier_ma_arr_delay_origin,carrier_ma_arr_delay_dest
0,2019-01-01,1,1,UA,ZW,3873,12217,HSV,"Huntsville, AL",13930,ORD,"Chicago, IL",1850,1847.0,-3.0,2045,2104.0,19.0,115.0,137.0,86.0,1,510,43.0,0.00,0.0,0.0,4.3,99.0,0.00,0.3,0.0,4.7,1711.0,108.0,2000,1800,25,1221713930,11276.000000,361.4,361.4,689.2,361.4,689.2
1,2019-01-01,1,1,WN,WN,1458,12206,HRL,"Harlingen/San Benito, TX",12191,HOU,"Houston, TX",2000,1957.0,-3.0,2100,2058.0,0.0,60.0,61.0,40.0,1,277,56.0,0.01,0.0,0.0,9.1,39.0,0.00,0.0,0.0,4.1,102.0,132.6,2000,1900,22,1220612191,91991.666667,114.4,114.4,119.8,114.4,119.8
2,2019-01-01,1,1,AA,AA,2561,14635,RSW,"Fort Myers, FL",13930,ORD,"Chicago, IL",1732,1758.0,26.0,1955,2037.0,42.0,203.0,219.0,161.0,1,1120,28.0,0.00,0.0,0.0,4.2,99.0,0.00,0.3,0.0,4.7,1711.0,73.0,1900,1700,1,1463513930,119584.000000,23.2,23.2,88.8,23.2,88.8
3,2019-01-01,1,1,B6,B6,176,14843,SJU,"San Juan, PR",10529,BDL,"Hartford, CT",1333,1328.0,-5.0,1637,1642.0,5.0,244.0,254.0,224.0,1,1666,59.0,0.00,0.0,0.0,11.1,47.0,0.15,0.0,0.0,4.7,125.8,91.4,1600,1300,4,1484310529,57854.000000,61.8,61.8,95.0,61.8,95.0
4,2019-01-01,1,1,AA,AA,2798,13930,ORD,"Chicago, IL",14771,SFO,"San Francisco, CA",1700,1750.0,50.0,1954,2025.0,31.0,294.0,275.0,246.0,1,1846,99.0,0.00,0.3,0.0,4.7,0.0,0.00,0.0,0.0,6.3,496.6,2390.6,1900,1600,1,1393014771,149265.166667,294.0,294.0,59.8,294.0,59.8


In [86]:
df.drop(['carrier_ma_arr_delay_x', 'carrier_ma_arr_delay_y', 'carrier_ma_arr_delay'], axis = 1, inplace = True)

In [96]:
df_ma_hourly_delay_origin = df[[
    'origin_airport_id', 'dep_time_binned', 'day', 'arr_delay']]
df_ma_hourly_delay_origin = df_ma_hourly_delay_origin.groupby(
    ['day', 'origin_airport_id', 'dep_time_binned']).sum().reset_index()
df_ma_hourly_delay_origin.fillna(0, inplace=True)

df_ma_hourly_delay_origin['ma_hourly_delay_origin'] = df_ma_hourly_delay_origin['arr_delay'].rolling(
    2).mean()
df_ma_hourly_delay_origin.fillna(0, inplace=True)
df_ma_hourly_delay_origin.drop('arr_delay', axis=1, inplace=True)

df = pd.merge(df, df_ma_hourly_delay_origin, how='left', on=[
              'origin_airport_id', 'day', 'dep_time_binned'])

In [97]:
df_ma_hourly_delay_dest = df[['dest_airport_id',
                              'dep_time_binned', 'day', 'arr_delay']]
df_ma_hourly_delay_dest = df_ma_hourly_delay_dest.groupby(
    ['day', 'dest_airport_id', 'dep_time_binned']).sum().reset_index()
df_ma_hourly_delay_dest.fillna(0, inplace=True)

df_ma_hourly_delay_dest['ma_hourly_delay_dest'] = df_ma_hourly_delay_dest['arr_delay'].rolling(
    2).mean()
df_ma_hourly_delay_dest.fillna(0, inplace=True)
df_ma_hourly_delay_dest.drop('arr_delay', axis=1, inplace=True)


df = pd.merge(df, df_ma_hourly_delay_dest, how='left', on=[
              'dest_airport_id', 'day', 'dep_time_binned'])

In [98]:
df_route = df[['encoded_op_unique_carrier', 'route', 'arr_delay', 'day']]

df_route = df_route.groupby(['encoded_op_unique_carrier', 'day','route']).sum().reset_index()
df_route['carrier_route_delay'] = df_route['arr_delay'].rolling(5).mean()

df_route.fillna(0, inplace=True)
df_route.drop('arr_delay', axis =1, inplace=True)

df = pd.merge(df, df_route, how='left', on=['encoded_op_unique_carrier','route','day'])


In [99]:
df.head()

,fl_date,day,week,mkt_unique_carrier,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,origin_cloud_cover,origin_precipitation,origin_snow_depth,origin_snow_fall,origin_wind_speed,dest_cloud_cover,dest_precipitation,dest_snow_depth,dest_snow_fall,dest_wind_speed,dest_ma_arr_delay,origin_ma_arr_delay,arr_time_binned,dep_time_binned,encoded_op_unique_carrier,route,annual_passengers_route,carrier_ma_arr_delay_origin,carrier_ma_arr_delay_dest,ma_hourly_delay_origin_x,ma_hourly_delay_origin_y,ma_hourly_delay_dest,carrier_route_delay
0,2019-01-01,1,1,UA,ZW,3873,12217,HSV,"Huntsville, AL",13930,ORD,"Chicago, IL",1850,1847.0,-3.0,2045,2104.0,19.0,115.0,137.0,86.0,1,510,43.0,0.00,0.0,0.0,4.3,99.0,0.00,0.3,0.0,4.7,1711.0,108.0,2000,1800,25,1221713930,11276.000000,361.4,689.2,9.5,9.5,1548.5,361.4
1,2019-01-01,1,1,WN,WN,1458,12206,HRL,"Harlingen/San Benito, TX",12191,HOU,"Houston, TX",2000,1957.0,-3.0,2100,2058.0,0.0,60.0,61.0,40.0,1,277,56.0,0.01,0.0,0.0,9.1,39.0,0.00,0.0,0.0,4.1,102.0,132.6,2000,1900,22,1220612191,91991.666667,114.4,119.8,0.0,0.0,65.0,35.0
2,2019-01-01,1,1,AA,AA,2561,14635,RSW,"Fort Myers, FL",13930,ORD,"Chicago, IL",1732,1758.0,26.0,1955,2037.0,42.0,203.0,219.0,161.0,1,1120,28.0,0.00,0.0,0.0,4.2,99.0,0.00,0.3,0.0,4.7,1711.0,73.0,1900,1700,1,1463513930,119584.000000,23.2,88.8,63.5,63.5,1753.0,11.4
3,2019-01-01,1,1,B6,B6,176,14843,SJU,"San Juan, PR",10529,BDL,"Hartford, CT",1333,1328.0,-5.0,1637,1642.0,5.0,244.0,254.0,224.0,1,1666,59.0,0.00,0.0,0.0,11.1,47.0,0.15,0.0,0.0,4.7,125.8,91.4,1600,1300,4,1484310529,57854.000000,61.8,95.0,21.0,21.0,2.5,5.6
4,2019-01-01,1,1,AA,AA,2798,13930,ORD,"Chicago, IL",14771,SFO,"San Francisco, CA",1700,1750.0,50.0,1954,2025.0,31.0,294.0,275.0,246.0,1,1846,99.0,0.00,0.3,0.0,4.7,0.0,0.00,0.0,0.0,6.3,496.6,2390.6,1900,1600,1,1393014771,149265.166667,294.0,59.8,890.5,890.5,79.0,41.4


In [100]:
# Export

df.to_csv("moving_averages_features.csv")